In [2]:
# Tugas Besar IF3270 Pembelajaran Mesin
# Anggota Kelompok:
# 1. Gede Prasidha Bhawarnawa - 13520004
# 2. Shadiq Harwiz - 13520038
# 3. Dwi Kalam Amal Tauhid - 13519210
# 4. Rahmat Rafid Akbar - 13520090

# Section Tugas Besar A: Implementasi Forward Propagation untuk Feed Forward Neural Network (FFNN)

# Import necessary libraries
import numpy as np

In [3]:
# Softmax Activation Function Implementation
def softmax(arr):
    numerator = np.exp(arr)
    denominator = np.sum(numerator)
    return numerator / denominator

In [4]:
# Sigmoid Activation Function Implementation
def sigmoid(arr):
    numerator = np.exp(arr)
    denominator = numerator + 1
    return numerator / denominator

In [5]:
# Linear Activation Function Implementation
def linear(arr):
    return arr

In [7]:
# Rectified Linear Unit (ReLU) Activation Function Implementation
def relu(arr):
    for i in range(len(arr)):
        if arr[i] < 0:
            arr[i] = 0
    return arr

arr = np.array([-1, 2, 3, 4, 5])
print(arr)
arr2 = relu(arr)
print(arr2)

[0 2 3 4 5]
[0 2 3 4 5]
